# Imports

In [14]:
import os
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import scipy as sp
import scipy.signal
import scipy.stats
from scipy.stats.mstats import gmean

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from scipy.stats import mode
from sklearn.model_selection import StratifiedKFold, GridSearchCV, LeaveOneOut

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

from utils import Dataset, variance_thresholding, standardize, mcc, calculate_metrics, calculate_metrics_statistics

In [15]:
# parameters for Welch's method for estimating power spectrum

NPERSEG = 13                    # length of segment
NOVERLAP = int(0.75 * NPERSEG)  # overlap of segments
NFFT = NPERSEG                  # length of FFT
WINDOW = "hann"                 # window function type

# parameters for saving data
PROCESSED_DATA_DIR = "processed_data24h"
DEPRESJON_PREFIX = "manual_depresjon24h"
HYPERAKTIV_PREFIX = "manual_hyperaktiv24h"
PSYKOSE_PREFIX = "manual_psykose24h"

# Manual feature extraction

## Helper functions

In [16]:
def basic_data_cleaning(data: List[pd.DataFrame]) -> List[pd.DataFrame]:
    """
    Assumes DataFrames with "timestamp", "date" and "activity" columns.
    
    Performs cleaning operations:
    - assure format YYYY-MM-DD HH:MM:SS for "timestamp"
    - drop redundant "date" column
    - assure float32 format for "activity"
    
    :param data: list of DataFrames
    :returns: list of cleaned DataFrames
    """
    data = [df.copy() for df in data]  # create copy to avoid side effects
    
    for df in data:
        df["timestamp"] = pd.to_datetime(df["timestamp"],
                                         format="%Y-%m-%d %H:%M:%S")
        df.drop("date", axis=1, inplace=True)
        df["activity"] = df["activity"].astype(np.float32)
    
    return data


def get_day_part(df: pd.DataFrame, part: str) -> pd.DataFrame:
    """
    For given DataFrame with "timestamp" column returns only those rows that
    correspond to the chosen part of day.
    
    Parts are "day" and "night", defined as:
    - "day": [8:00, 21:00)
    - "night": [21:00, 8:00)
    
    :param df: DataFrame to select rows from
    :param part: part of day, either "day" or "night"
    :returns: DataFrame, subset of rows of df
    """
    if part == "day":
        df = df.loc[(df["timestamp"].dt.hour >= 8) &
                    (df["timestamp"].dt.hour < 21)]
    elif part == "night":
        df = df.loc[(df["timestamp"].dt.hour >= 21) |
                    (df["timestamp"].dt.hour < 8)]
    else:
        raise ValueError(f'Part should be "day" or "night", got "{part}"')
        
    return df


def fill_missing_activity(df: pd.DataFrame) -> pd.DataFrame:
    """
    Makes sure that "timestamp" column has minute resolution with no missing
    values from start to end and replaces all NaNs in "activity" column with
    mean average value.
    
    :param data: DataFrame with "timestamp" and "activity" columns
    :returns: cleaned DataFrame
    """
    df = df.copy()  # create copy to avoid side effects
    
    # resample to the basic frequency, i.e. minute; this will create NaNs for
    # any rows that may be missing
    df = df.resample("min", on="timestamp").mean()
    
    # recreate index and "timestamp" column
    df = df.reset_index()
    
    # fill any NaNs with mean activity value
    df["activity"] = df["activity"].fillna(df["activity"].mean())

    return df


def resample(df: pd.DataFrame, freq: str = "H") -> pd.DataFrame:
    """
    Resamples time series DataFrame with given frequency, aggregating each
    segment with a mean.

    :param df: DataFrame with "timestamp" and "activity" columns
    :param freq: resampling frequency passed to Pandas resample() function
    :returns: DataFrame with "timestamp" and "activity" columns
    """
    df = df.copy()  # create copy to avoid side effects
    
    # group with given frequency
    df = df.resample(freq, on="timestamp").mean()

    # recreate "timestamp" column
    df = df.reset_index()

    return df


def proportion_of_zeros(x: np.ndarray) -> float:
    """
    Calculates proportion of zeros in given array, i.e. number of zeros divided
    by length of array.
    
    :param x: 1D Numpy array
    :returns: proportion of zeros
    """
    # we may be dealing with floating numbers, we can't use direct comparison
    zeros_count = np.sum(np.isclose(x, 0))
    return zeros_count / len(x)


def power_spectral_density(df: pd.DataFrame) -> np.ndarray:
    """
    Calculates power spectral density (PSD) from "activity" column of a
    DataFrame.
    
    :param df: DataFrame with "activity" column
    :returns: 1D Numpy array with power spectral density
    """
    psd = scipy.signal.welch(
        x=df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="density"
    )[1]
    return psd


def spectral_flatness(df: pd.DataFrame) -> float:
    """
    Calculates spectral flatness of a signal, i.e. a geometric mean of the
    power spectrum divided by the arithmetic mean of the power spectrum.
    
    If some frequency bins in the power spectrum are close to zero, they are
    removed prior to calculation of spectral flatness to avoid calculation of
    log(0).
    
    :param df: DataFrame with "activity" column
    :returns: spectral flatness value
    """
    power_spectrum = scipy.signal.welch(
        df["activity"].values,
        fs=(1/13),
        nperseg=11,
        noverlap=10,
        nfft=NFFT,
        window=WINDOW,
        scaling="spectrum"
    )[1]
    
    non_zeros_mask = ~np.isclose(power_spectrum, 0)
    power_spectrum = power_spectrum[non_zeros_mask]
    
    return scipy.stats.gmean(power_spectrum) / power_spectrum.mean()

## Feature extraction

In [17]:
def extract_time_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in time domain.
    
    :param df_resampled: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = df["activity"].values
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X, ddof=1),  # apply Bessel's correction
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "proportion_of_zeros": proportion_of_zeros(X)
    }
    
    return pd.DataFrame([features])

In [18]:
def extract_frequency_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts features from activity signal in frequency domain, i.e. calculated
    from its Power Spectral Density (PSD).
    
    :param df: DataFrame with "activity" column
    :returns: DataFrame with a single row representing features
    """
    X = power_spectral_density(df)
    
    features = {
        "minimum": np.min(X),
        "maximum": np.max(X),
        "mean": np.mean(X),
        "median": np.median(X),
        "variance": np.var(X),
        "kurtosis": sp.stats.kurtosis(X),
        "skewness": sp.stats.skew(X),
        "coeff_of_var": sp.stats.variation(X),
        "iqr": sp.stats.iqr(X),
        "trimmed_mean": sp.stats.trim_mean(X, proportiontocut=0.1),
        "entropy": sp.stats.entropy(X, base=2),
        "spectral_flatness": spectral_flatness(df)
    }
    
    return pd.DataFrame([features])

In [19]:
def extract_features_for_dataframes(dfs: List[pd.DataFrame], freq: str = "H") \
        -> Dict[str, pd.DataFrame]:
    """
    Calculates time and frequency features for given DataFrames. Uses given
    frequency for resampling.
    
    Calculates features separately for:
    - full 13hs
    
    :param dfs: list of DataFrames to extract features from; each one has to
    have "timestamp" and "activity" columns
    :param freq: resampling frequency
    :returns: dictionary with keys "day", corresponding to features from given parts of day
    """
    full_dfs = basic_data_cleaning(dfs)
    full_dfs = [fill_missing_activity(df) for df in full_dfs]
    full_dfs = [resample(df, freq=freq) for df in full_dfs]

    datasets = {}
    
    for part, list_of_dfs in [("day", full_dfs)]:
        features = []
        for df in list_of_dfs:
            time_features = extract_time_features(df)
            freq_features = extract_frequency_features(df)

            merged_features = pd.merge(
                time_features,
                freq_features,
                left_index=True,
                right_index=True,
                suffixes=["_time", "_freq"]
            )
            features.append(merged_features)

        datasets[part] = pd.concat(features)
        datasets[part].reset_index(drop=True, inplace=True)
    
    return datasets



## Depresjon

In [20]:
dataset = Dataset(dirpath=os.path.join("data_day", "depresjon"))
condition = dataset.condition
control = dataset.control

In [21]:
condition[0]

,timestamp,date,activity
0,2003-06-20 08:00:00,2003-06-20,0
1,2003-06-20 08:01:00,2003-06-20,0
2,2003-06-20 08:02:00,2003-06-20,0
3,2003-06-20 08:03:00,2003-06-20,0
4,2003-06-20 08:04:00,2003-06-20,0
...,...,...,...
775,2003-06-20 20:55:00,2003-06-20,173
776,2003-06-20 20:56:00,2003-06-20,273
777,2003-06-20 20:57:00,2003-06-20,81
778,2003-06-20 20:58:00,2003-06-20,0


In [22]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]
    
    entire_df = pd.concat([condition_df, control_df], ignore_index=True)
    
    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [23]:
for part, df in datasets.items():
    filename = f"{DEPRESJON_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [24]:
condition_df

,minimum_time,maximum_time,mean_time,median_time,variance_time,kurtosis_time,skewness_time,coeff_of_var_time,iqr_time,trimmed_mean_time,...,mean_freq,median_freq,variance_freq,kurtosis_freq,skewness_freq,coeff_of_var_freq,iqr_freq,trimmed_mean_freq,entropy_freq,spectral_flatness
0,1.550000,691.683350,182.697449,133.149994,37141.070312,1.777838,1.535081,1.013476,108.100006,152.893936,...,1.591745e+06,7.676033e+05,1.475475e+12,-1.050967,0.848570,0.763120,1.622347e+06,1.591745e+06,2.424109,0.762552
1,0.133333,166.016663,74.126923,76.199997,3241.548584,-1.376535,0.160101,0.737937,88.883329,72.500000,...,8.366959e+04,4.784316e+04,6.995833e+09,-1.160195,0.652128,0.999659,1.314086e+05,8.366959e+04,2.057472,0.459421
2,3.733333,375.133331,152.155136,178.149994,19131.087891,-1.252746,0.325477,0.873378,203.799994,145.377274,...,3.698682e+05,1.741404e+05,1.241842e+11,-0.690863,1.051530,0.952766,3.371126e+05,3.698682e+05,2.240067,0.670169
3,12.383333,1146.166626,514.406372,337.850006,220003.859375,-1.666639,0.264552,0.876048,946.816689,502.612122,...,3.252897e+06,2.089687e+06,1.106185e+13,0.044950,1.178468,1.022453,3.526273e+06,3.252897e+06,2.133224,0.571466
4,2.000000,332.100006,121.837173,79.349998,11930.444336,-0.746519,0.778118,0.861326,147.866669,113.616669,...,2.007052e+05,1.341730e+05,3.714222e+10,0.042982,1.116005,0.960230,2.100331e+05,2.007052e+05,2.190617,0.575674
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354,0.000000,476.649994,89.555122,15.933333,22290.072266,1.731359,1.751707,1.601711,89.383333,62.506058,...,9.399210e+05,9.031073e+05,4.646245e+11,-1.358720,0.325193,0.725203,1.111502e+06,9.399210e+05,2.398015,0.683030
355,24.033333,527.049988,247.498718,192.733337,25436.404297,-0.625517,0.704588,0.619119,124.633347,242.400009,...,2.409708e+05,7.242294e+04,5.466909e+10,-1.379489,0.635671,0.970301,3.797765e+05,2.409708e+05,2.130542,0.548743
356,100.699997,374.566681,211.474350,188.366669,6349.609863,-0.357352,0.765957,0.362022,72.466660,206.718170,...,2.092595e+05,1.759142e+05,2.411453e+10,-1.284133,0.302514,0.742086,2.460169e+05,2.092595e+05,2.359982,0.598252
357,0.316667,474.816681,222.148743,191.766663,26151.658203,-1.392443,0.199899,0.699399,291.650017,219.345444,...,7.296481e+05,4.260060e+05,5.777451e+11,-1.093494,0.849756,1.041729,9.943926e+05,7.296481e+05,2.066749,0.516538


In [25]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"depresjon_y.csv")
y.to_csv(filepath, header=False, index=False)

In [26]:
# to get ensemble:
import os

def sort_key(filename):
    parts = filename.split("_")
    # Wydobycie numerów jako liczb całkowitych
    condition_number = int(parts[1])  # Druga część np. "1" z "condition_1"
    segment_number = int(parts[3].split(".")[0])  # Trzecia część np. "1" z "segmen1.csv"
    return (condition_number, segment_number)


In [27]:
# Removing NaNs
folder_control = './data_day/depresjon/control'
folder_condition = './data_day/depresjon/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["day"].index

mask = datasets["day"].notna().all(axis=1)
datasets["day"] = datasets["day"][mask]
y = y[mask]
z_depresjon = z[mask]

## Psykose

In [28]:
dataset = Dataset(dirpath=os.path.join("data_day", "psykose"))
condition = dataset.condition
control = dataset.control

In [29]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [30]:
for part, df in datasets.items():
    filename = f"{PSYKOSE_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [31]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"psykose_y.csv")
y.to_csv(filepath, header=False, index=False)
#datasets["full_24h"].dropna(inplace=True)

In [32]:
# Removing NaNs
folder_control = './data_day/psykose/control'
folder_condition = './data_day/psykose/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["day"].index
mask = datasets["day"].notna().all(axis=1)
datasets["day"] = datasets["day"][mask]
y = y[mask]
z_psykose = z[mask]

Hyperactive

In [33]:
dataset = Dataset(dirpath=os.path.join("data_day", "hyperaktiv"))
condition = dataset.condition
control = dataset.control

In [34]:
condition_parts_dfs = extract_features_for_dataframes(condition, freq="H")
control_parts_dfs = extract_features_for_dataframes(control, freq="H")

datasets = {}

for part in ["day"]:
    condition_df = condition_parts_dfs[part]
    control_df = control_parts_dfs[part]

    entire_df = pd.concat([condition_df, control_df], ignore_index=True)

    # Przypisujemy wynik do słownika datasets
    datasets[part] = entire_df

In [35]:

for part, df in datasets.items():
    filename = f"{HYPERAKTIV_PREFIX}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    df.to_csv(filepath, index=False)

In [36]:
y = np.concatenate((np.ones(len(condition)), np.zeros(len(control))))
y = pd.Series(y, dtype=int)

filepath = os.path.join(PROCESSED_DATA_DIR, f"hyperaktiv_y.csv")
y.to_csv(filepath, header=False, index=False)

In [37]:
# Removing NaNs
folder_control = './data_day/hyperaktiv/control'
folder_condition = './data_day/hyperaktiv/condition'
z_control = []
z_condition = []
for filename in sorted(os.listdir(folder_control), key=sort_key):
    z_control.append(filename.split("_")[0] + filename.split("_")[1])
    
for filename in sorted(os.listdir(folder_condition), key=sort_key):
    z_condition.append(filename.split("_")[0] + filename.split("_")[1])
    
z_control = pd.Series(z_control)
z_condition = pd.Series(z_condition)

z = pd.concat([z_condition, z_control])
z.index = datasets["day"].index
mask = datasets["day"].notna().all(axis=1)
datasets["day"] = datasets["day"][mask]
y = y[mask]
z_hyperaktiv = z[mask]

# Classification

## Classifiers, parameters, constants

In [38]:
classifiers = {
    "LR": LogisticRegression(
        penalty="elasticnet",
        random_state=0,
        solver="saga",
        max_iter=500
    ),
    "SVM": SVC(
        kernel="rbf",
        cache_size=512
    ),
    "RF": RandomForestClassifier(
        n_estimators=200,
        criterion="entropy"
    ),
    "LGBM": LGBMClassifier(
        n_estimators=200,
        verbosity=-1,
        random_state=0
    ),
    "XGB": XGBClassifier(
        n_estimators=200,
        random_state=0 
    )
}


param_grids = {
    "LR": {
        "C": [0.001, 0.01, 0.1, 0.5, 1, 2, 5, 10, 25, 50, 100, 500, 1000],
        "class_weight": ["balanced"],
        "l1_ratio": [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5,
                     0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 1],
        "max_iter": [1000]
    },
    "SVM": {
        "C": [1, 10],
        "gamma": ["scale"],
        "class_weight": [None, "balanced"]
    },
    "RF": {
        "class_weight": [None, "balanced", "balanced_subsample"]
    },
    "LGBM": {
        "num_leaves": [31, 50],
        "min_child_samples": [10, 20],
        "class_weight": [None, "balanced"]
    },
    "XGB": {
        "learning_rate": [0.01, 0.1],
        "max_depth": [3, 6],
        "subsample": [0.8, 1.0],
        "colsample_bytree": [0.8, 1.0],
        "scale_pos_weight": [1, 10]
    }
}

## Classification

# Classification - Depresjon

In [26]:
dataset = DEPRESJON_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [27]:
# Assuming PROCESSED_DATA_DIR and y_filename are already defined
datasets = {}

# Load datasets
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

# Sprawdzamy kształt danych po usunięciu NaN
#print(datasets['full_24h'].shape)
#print(y.shape)
#print(z_depresjon.shape)

In [28]:
print("Wyniki dla Depresjon:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_depresjon.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Depresjon:
PART: day
  LR
    accuracy: 0.4685 +- 0.0607
    balanced_accuracy: 0.5103 +- 0.0398
    f1: 0.5443 +- 0.0182
    precision: 0.4307 +- 0.0367
    recall: 0.7636 +- 0.1078
    specificity: 0.2571 +- 0.1744
    ROC_AUC: 0.5103 +- 0.0398
    MCC: 0.0159 +- 0.0887

  SVM
    accuracy: 0.5824 +- 0.0012
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.0000 +- 0.0000
    precision: 1.0000 +- 0.0000
    recall: 0.0000 +- 0.0000
    specificity: 1.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  RF
    accuracy: 0.5713 +- 0.0335
    balanced_accuracy: 0.5229 +- 0.0265
    f1: 0.3026 +- 0.0605
    precision: 0.4956 +- 0.0987
    recall: 0.2289 +- 0.0674
    specificity: 0.8169 +- 0.0892
    ROC_AUC: 0.5229 +- 0.0265
    MCC: 0.0641 +- 0.0757

  LGBM
    accuracy: 0.6262 +- 0.0175
    balanced_accuracy: 0.5762 +- 0.0158
    f1: 0.3760 +- 0.0404
    precision: 0.6322 +- 0.0672
    recall: 0.2723 +- 0.0451
    specificity: 0.8800 +- 0.0478
    ROC_AU

# Classification - Psykose

In [29]:
dataset = PSYKOSE_PREFIX  # DEPRESJON_PREFIX or PSYKOSE_PREFIX
y_filename = "depresjon_y.csv" if dataset == DEPRESJON_PREFIX else "psykose_y.csv"

In [30]:
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [31]:
print("Wyniki dla Psykose:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_psykose.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Psykose:
PART: day
  LR
    accuracy: 0.4067 +- 0.0014
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.5782 +- 0.0014
    precision: 0.4067 +- 0.0014
    recall: 1.0000 +- 0.0000
    specificity: 0.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  SVM
    accuracy: 0.5933 +- 0.0014
    balanced_accuracy: 0.5000 +- 0.0000
    f1: 0.0000 +- 0.0000
    precision: 1.0000 +- 0.0000
    recall: 0.0000 +- 0.0000
    specificity: 1.0000 +- 0.0000
    ROC_AUC: 0.5000 +- 0.0000
    MCC: 0.0000 +- 0.0000

  RF
    accuracy: 0.5785 +- 0.0147
    balanced_accuracy: 0.4935 +- 0.0192
    f1: 0.6611 +- 0.4174
    precision: 0.2000 +- 0.2449
    recall: 0.0377 +- 0.0555
    specificity: 0.9494 +- 0.0261
    ROC_AUC: 0.4935 +- 0.0192
    MCC: -0.0565 +- 0.0925

  LGBM
    accuracy: 0.6234 +- 0.0236
    balanced_accuracy: 0.5443 +- 0.0281
    f1: 0.1996 +- 0.0925
    precision: 0.7762 +- 0.1961
    recall: 0.1203 +- 0.0655
    specificity: 0.9683 +- 0.0284
    ROC_AUC

# Classification - Hyperactive

In [39]:
dataset = HYPERAKTIV_PREFIX
y_filename = "hyperaktiv_y.csv"

In [40]:
for part in ["day"]:
    filename = f"{dataset}_{part}.csv"
    filepath = os.path.join(PROCESSED_DATA_DIR, filename)
    datasets[part] = pd.read_csv(filepath, header=0).values

# Load y values
y = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, y_filename), header=None, dtype=int)
y = y.values.ravel()
# Usuwanie wierszy, które zawierają NaN w danych
for part in datasets:
    # Indeksy wierszy, które zawierają NaN w dowolnej kolumnie
    nan_indices = np.isnan(datasets[part]).any(axis=1)
    
    # Usuwamy te wiersze z datasets i y
    datasets[part] = datasets[part][~nan_indices]
    y = y[~nan_indices]

In [41]:
print("Wyniki dla Hyperaktiv:")
for part in ["day"]:
    print(f"PART: {part}")
    
    X = datasets[part]
    for clf_type in ["LR", "SVM", "RF", "LGBM", "XGB"]:
        print(f"  {clf_type}")
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
        
        test_scores = []
        group_results = {}  
        
        for train_idx, test_idx in folds.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx] 
            y_train, y_test = y[train_idx], y[test_idx]
            
            #X_train, X_test = variance_thresholding(X_train, X_test, threshold=0.05)
            X_train, X_test = standardize(X_train, X_test)
            
            grid_search = GridSearchCV(
                estimator=classifiers[clf_type], 
                param_grid=param_grids[clf_type], 
                scoring="f1_weighted",
                n_jobs=-1,
                refit=True,
                #cv=LeaveOneOut()
            )
            grid_search.fit(X_train, y_train)

            clf = grid_search.best_estimator_
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            for i in range(len(test_idx)):
                
                condition_id = z_hyperaktiv.iloc[test_idx[i]] 
                class_label = y[test_idx[i]] 
                unique_id = (condition_id, class_label)  

                if unique_id not in group_results:
                    group_results[unique_id] = []  
                group_results[unique_id].append(y_pred[i])  
            final_predictions = {}
            for unique_id, preds in group_results.items():
                condition_id, class_label = unique_id
                majority_vote = mode(preds).mode[0]
                final_predictions[unique_id] = majority_vote 
            y_true_grouped = []
            y_pred_grouped =  []
            for k,v in final_predictions.items():
                y_true_grouped.append(k[1])
                y_pred_grouped.append(v)
            metrics = calculate_metrics(y_true_grouped, y_pred_grouped)
            test_scores.append(metrics)
            
        final_scores = calculate_metrics_statistics(test_scores)
        
        for metric, (mean, stddev) in final_scores.items():
            print(f"    {metric}: {mean:.4f} +- {stddev:.4f}")
        
        print()

Wyniki dla Hyperaktiv:
PART: day
  LR


/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3.8/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/anaconda3/envs/python38/lib/python3

    accuracy: 0.5389 +- 0.0195
    balanced_accuracy: 0.5401 +- 0.0209
    f1: 0.5420 +- 0.0111
    precision: 0.5644 +- 0.0290
    recall: 0.5233 +- 0.0234
    specificity: 0.5570 +- 0.0552
    ROC_AUC: 0.5401 +- 0.0209
    MCC: 0.0805 +- 0.0419

  SVM
    accuracy: 0.5277 +- 0.0416
    balanced_accuracy: 0.5285 +- 0.0416
    f1: 0.5356 +- 0.0417
    precision: 0.5514 +- 0.0452
    recall: 0.5233 +- 0.0515
    specificity: 0.5338 +- 0.0723
    ROC_AUC: 0.5285 +- 0.0416
    MCC: 0.0569 +- 0.0834

  RF
    accuracy: 0.5143 +- 0.0392
    balanced_accuracy: 0.5148 +- 0.0402
    f1: 0.5265 +- 0.0309
    precision: 0.5394 +- 0.0458
    recall: 0.5180 +- 0.0423
    specificity: 0.5117 +- 0.0951
    ROC_AUC: 0.5148 +- 0.0402
    MCC: 0.0292 +- 0.0813

  LGBM
    accuracy: 0.5634 +- 0.0234
    balanced_accuracy: 0.5659 +- 0.0245
    f1: 0.5530 +- 0.0228
    precision: 0.5975 +- 0.0385
    recall: 0.5199 +- 0.0503
    specificity: 0.6120 +- 0.0830
    ROC_AUC: 0.5659 +- 0.0245
    MCC: 0.1336 +